In [18]:
# 모듈 불러오기
import warnings
warnings.filterwarnings(action='ignore')

import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# 경로 설정
JANUARY = './data/temp_1월_명사추출.bin'

In [5]:
# 데이터 로드
with open(JANUARY, 'rb') as f:
    data_jan = pickle.load(f)

In [9]:
# 데이터 확인
data_jan.head(3)

,상품군,홈쇼핑모아링크,상품명,방송시작시간,방송종료시간,이미지리스트,줄글,clean_text,doc,doc_nouns
0,패션·잡화,https://hsmoa.com/i?id=8788275&from=web_timeline,[삼성카드5%할인][NS Shop+]그렉노먼 남성 구스다운 롱 벤치코트,2019-01-01 00:02:00,2019-01-01 01:02:00,[http://image.nsmall.com/ec_descimages/4/18/12...,\n 제품소재■혼용율\n겉감: 폴리에스터 100%안감: 폴리에스터 100%충전재1...,"[GREGNORMAN, 2018, F/W, 그렉노먼, |구스다운, 롱벤치코트, GO...",GREGNORMAN 2018 F/W 그렉노먼 |구스다운 롱벤치코트 GOOSE DOW...,"[그렉, 노먼, 스, 다운, 롱, 벤치, 코트, 남성, 용, 종, 블랙, 차콜, 그..."
1,패션·잡화,https://hsmoa.com/i?id=8788197&from=web_timeline,[삼성카드5%할인][NS Shop+]아리스토우 남성기모티셔츠5종,2019-01-01 01:02:00,2019-01-01 02:00:00,[http://image.nsmall.com/ec_descimages/7/18/12...,"\n 제품소재■혼용율\n폴리에스터 90%, 폴리우레탄 10%(5종동일)색상차콜그레...","[아리스토우, 전속모델, 김재원, Aristow, ELITE, SPORTS, STO...",아리스토우 전속모델 김재원 Aristow ELITE SPORTS STORIES 아리...,"[리스, 토우, 전속, 모델, 김재원, 아리, 스토, 귀족, 뜻, 프랑스어, 엘리트..."
2,화장품·미용,https://hsmoa.com/i?id=8785322&from=web_timeline,[삼성카드5%할인][NS Shop+]마리끌레르 파리컬렉션 리얼 하이드로 립스틱,2019-01-01 02:00:00,2019-01-01 03:00:00,[http://image.nsmall.com/ec_descimages/9/18/11...,\n 용량/중량립스틱 각 3.5g클러치 : 26.5*12.5*3cm(재질 : 합성...,"[marie, claire, PARIS, Rouge, de'artisan, 강렬한,...",marie claire PARIS Rouge de'artisan 강렬한 컬러와 MA...,"[컬러, 함유, 코어, 필링, 리얼, 이드, 립스틱, 현대, 여성, 위해, 글로벌,..."


# 명사만 선택해서 Tf-Idf

In [21]:
# 명사 비어 있으면 제외
def change_null_str(x):
    if len(x) == 0:
        return np.nan
    else:
        return x

data_jan['doc_nouns'] = data_jan['doc_nouns'].apply(lambda x: change_null_str(x))
data_jan.isnull().sum() # doc_nouns: 11
data_jan = data_jan.dropna().reset_index(drop=True)
data_jan.tail()

,상품군,홈쇼핑모아링크,상품명,방송시작시간,방송종료시간,이미지리스트,줄글,clean_text,doc,doc_nouns
683,가전·디지털,https://hsmoa.com/i?id=8825644&from=web_timeline,[NS Shop+](일시불)삼성 UHD TV UN55NU7010FXKR,2019-01-31 21:03:00,2019-01-31 22:03:00,[http://image.nsmall.com/ec_descimages/6/19/01...,\n 품명/모델명TV : 삼성 UHDTV UN55NU7010FXKR사운드바:삼성 ...,"[제품, 주요, 특장점, 4K, UHD, TV, 전, 모델, 화질, 선명도, 95%...",제품 주요 특장점 4K UHD TV 전 모델 화질 선명도 95% UHD 방송 시대에...,"[제품, 주요, 특, 장점, 전, 모델, 화질, 선명, 방송, 시대, 걸, 삼성, ..."
684,가전·디지털,https://hsmoa.com/i?id=8825656&from=web_timeline,[NS Shop+](일시불)삼성 유로피안 냉장고 RS82M6000S8,2019-01-31 22:03:00,2019-01-31 23:03:00,[http://image.nsmall.com/ec_descimages/1/19/01...,\n 품명/모델명냉장고 : 삼성 양문형 냉장고 RS82M6000S8공청기 : 삼성...,"[감각적인, 주방의, 완성, 도어와, 하나, 된, 듯, 슬림하게, 디자인된, 히든,...",감각적인 주방의 완성 도어와 하나 된 듯 슬림하게 디자인된 히든 핸들과 고급스러운 ...,"[감각, 주방, 완성, 도어, 하나, 듯, 슬림하, 디자인, 히든, 핸들, 메탈, ..."
685,가전·디지털,https://hsmoa.com/i?id=8825651&from=web_timeline,[NS Shop+](일시불)LG 노트북 그램 15인치 15Z980-L.AR1DK,2019-01-31 23:03:00,2019-01-31 23:03:00,[http://image.nsmall.com/ec_descimages/8/19/01...,\n 품명/모델명■노트북 : LG 그램 15_파워팩15Z980-L.AR1DK■스피...,"[C, LG, 전자, ALLX, NEW, gram, 또, 한, 번의, 혁신, LG,...",C LG 전자 ALLX NEW gram 또 한 번의 혁신 LG 72Wh] 대용량 7...,"[전자, 또, 번의, 혁신, 용량, 초고, 밀도, 배터리, 양방, 항, 부스터, 충..."
686,가전·디지털,https://hsmoa.com/i?id=8827115&from=web_timeline,[NS Shop+](일시불)LG 노트북 그램 15인치 15Z980-L.AR1DK,2019-01-31 23:23:00,2019-01-31 23:23:00,[http://image.nsmall.com/ec_descimages/8/19/01...,\n 품명/모델명■노트북 : LG 그램 15_파워팩15Z980-L.AR1DK■스피...,"[C, LG, 전자, ALLX, NEW, gram, 또, 한, 번의, 혁신, LG,...",C LG 전자 ALLX NEW gram 또 한 번의 혁신 LG 72Wh] 대용량 7...,"[전자, 또, 번의, 혁신, 용량, 초고, 밀도, 배터리, 양방, 항, 부스터, 충..."
687,가전·디지털,https://hsmoa.com/i?id=8827116&from=web_timeline,[NS Shop+](일시불)LG 노트북 그램 15인치 15Z980-L.AR1DK,2019-01-31 23:43:00,2019-01-31 23:43:00,[http://image.nsmall.com/ec_descimages/8/19/01...,\n 품명/모델명■노트북 : LG 그램 15_파워팩15Z980-L.AR1DK■스피...,"[C, LG, 전자, ALLX, NEW, gram, 또, 한, 번의, 혁신, LG,...",C LG 전자 ALLX NEW gram 또 한 번의 혁신 LG 72Wh] 대용량 7...,"[전자, 또, 번의, 혁신, 용량, 초고, 밀도, 배터리, 양방, 항, 부스터, 충..."


In [22]:
# document 리스트
doc_nouns_list = []
for doc_nouns in data_jan['doc_nouns']:
    doc_nouns_list.append(" ".join(doc_nouns))

## Tf-Idf Vectorizer 파라미터

- `min_df`
    - 1

In [35]:
tfidf_matrix[2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [36]:
TfidfVectorizer?

In [30]:
# Tf-Idf Vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_features=500)
tfidf_matrix = tfidf_vectorizer.fit(doc_nouns)
(doc_nouns).toarray()
doc_similarities = np.dot(tfidf_matrix, tfidf_matrix.T)
doc_similarities

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])